In [69]:
import os
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas_datareader.data as web

# Main reference of Fina Project_1 https://medium.com/swlh/how-to-build-a-multi-factor-equity-portfolio-in-python-4560fab3df7b

In [70]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
plt.style.use('ggplot')
pd.options.mode.chained_assignment = None

fundamental_data = (pd.read_csv('SHARADAR_SF1_FUN.csv'))
equity_prices = (pd.read_csv('SHARADAR_SEP.csv'))
tickers_df = (pd.read_csv('SHARADAR_TICKERS.csv', low_memory=False))

In [71]:
tickers_df1 = tickers_df[tickers_df['location'].notnull()]
tickers_df1 = tickers_df1[tickers_df1['location'].str.contains("U.S.")]
tickers_df1 = (tickers_df1[['ticker', 'sector', 'name',
                 'industry', 'scalemarketcap']])
myset_ticker = set(tickers_df1.ticker)
list_tickers = list(myset_ticker)

USA_fundamentals = fundamental_data[fundamental_data['ticker'].isin(list_tickers)]

sector_stocks = tickers_df1[tickers_df1['sector'] != 'Real Estate']
sector_stocks = sector_stocks[sector_stocks['sector'] != 'Financial Services']
sector_tickers = sector_stocks['ticker'].tolist()

fundamentals = USA_fundamentals[USA_fundamentals.ticker.isin(sector_tickers)]
fundamentals = fundamentals[fundamentals.dimension == 'ART']

duplicateRowsDF = fundamentals[fundamentals.duplicated(['ticker', 'calendardate'])]
fundamentals = fundamentals.drop_duplicates(subset = ['ticker', 'calendardate'],\
                                            keep = 'first')
duplicateRowsDF = fundamentals[fundamentals.duplicated(['ticker', 'calendardate'])]


Data_for_Portfolio = fundamentals[fundamentals['marketcap'] >= 7e9]
tickers = Data_for_Portfolio['ticker'].tolist()
print('There are ' + str(len(set(tickers))) + ' tickers') #number of unique tickers

#create the dictionary with values and keys as dates
keys = tickers_df1['ticker']
values = tickers_df1['sector']
Dictionary_Sector_values = dict(zip(keys, values))

Data_for_Portfolio['sector'] = Data_for_Portfolio\
    ['ticker'].map(Dictionary_Sector_values)

There are 1000 tickers


## Portfolio Construction Starting Date

In [72]:
Data_for_Portfolio= Data_for_Portfolio[Data_for_Portfolio['calendardate'] > '2017-12-31']

## Constructing factors

In [73]:
### Value Factor ###
Data_for_Portfolio['E/P'] = Data_for_Portfolio['netinc'] / \
    Data_for_Portfolio['marketcap']
Data_for_Portfolio['EBITDA/EV'] = Data_for_Portfolio['ebitda'] / \
    Data_for_Portfolio['ev']
Data_for_Portfolio['FCF/P'] = Data_for_Portfolio['fcf'] / \
    Data_for_Portfolio['marketcap']

### Shareholder Yield ###
Data_for_Portfolio['Shareholder Yield'] = \
    -((Data_for_Portfolio['ncfdebt'] + \
       Data_for_Portfolio['ncfdiv'] + \
           Data_for_Portfolio['ncfcommon']) / Data_for_Portfolio['marketcap'])
    
####Long Term Business Strength
    
#Can you generate free cash flow
Data_for_Portfolio['FCF/Assets'] = Data_for_Portfolio['fcf'] / \
    Data_for_Portfolio['assets']

#Can you generate returns on investment   
Data_for_Portfolio['ROA'] = Data_for_Portfolio['roa']    
Data_for_Portfolio['ROIC'] = Data_for_Portfolio['roic']

#Do you have a defendable business model?
Data_for_Portfolio['GROSS MARGIN'] = Data_for_Portfolio['grossmargin']

#Current Financial Strength

Data_for_Portfolio['CURRENT RATIO'] = Data_for_Portfolio['currentratio']
Data_for_Portfolio['INTEREST/EBITDA'] = Data_for_Portfolio['intexp'] / \
                                        Data_for_Portfolio['ebitda']   

Data_for_Portfolio = Data_for_Portfolio.dropna()

Sector_stock_prices = equity_prices.loc  \
    [equity_prices['ticker'].isin(tickers)]

In [74]:
f_date = datetime.date(2018, 3, 31) 
l_date = datetime.date(2022, 6, 14) #choosing the last date, results in last
delta = l_date - f_date
quarters_delta = np.floor(delta.days/(365/4))
quarters_delta = int(quarters_delta)
first_quarter = str('2018-03-31') #using f_date
Data_for_Portfolio_master = pd.DataFrame(Data_for_Portfolio)

In [75]:
Winsorize_Threshold = .025 

Portfolio_Turnover = pd.DataFrame()
portfolio_returns = pd.DataFrame()

price_index = Sector_stock_prices.set_index('date')
price_index = price_index.index
price_index = price_index.unique()
price_index = pd.to_datetime(price_index)
price_index = price_index.sort_values()

In [76]:
Data_for_Portfolio_master_filter = pd.DataFrame()

for i in range(0, quarters_delta, 4):
    
    Date = pd.to_datetime(first_quarter) + pd.tseries.offsets.QuarterEnd(i)
    Date = Date.strftime('%Y-%m-%d')
    df=Data_for_Portfolio_master.loc[Data_for_Portfolio_master['calendardate'] == Date]
    Data_for_Portfolio_master_filter=Data_for_Portfolio_master_filter.append(df)


## Creating all the factors

In [77]:
###### VALUE FACTOR ######
    
#Winsorize the metric data and compress outliers if desired
Data_for_Portfolio_master_filter['E/P Winsorized'] = stats.mstats.winsorize(Data_for_Portfolio_master_filter['E/P'], limits=Winsorize_Threshold)
Data_for_Portfolio_master_filter['EBITDA/EV Winsorized'] = stats.mstats.winsorize(Data_for_Portfolio_master_filter['EBITDA/EV'],limits=Winsorize_Threshold)
Data_for_Portfolio_master_filter['FCF/P Winsorized'] =  stats.mstats.winsorize(Data_for_Portfolio_master_filter['FCF/P'],limits=Winsorize_Threshold)
 
#create Z score to normalize the metrics
Data_for_Portfolio_master_filter['E/P Z score'] = \
    stats.zscore(Data_for_Portfolio_master_filter['E/P Winsorized'])
Data_for_Portfolio_master_filter['EBITDA/EV Z score'] = \
    stats.zscore(Data_for_Portfolio_master_filter['EBITDA/EV Winsorized'])
Data_for_Portfolio_master_filter['FCF/P Z score'] = \
    stats.zscore(Data_for_Portfolio_master_filter['FCF/P Winsorized'])
    
Data_for_Portfolio_master_filter['Valuation Score'] = \
        Data_for_Portfolio_master_filter['E/P Z score'] \
        + Data_for_Portfolio_master_filter['EBITDA/EV Z score']\
        + Data_for_Portfolio_master_filter['FCF/P Z score']    


###### QUALITY FACTOR ######  

Data_for_Portfolio_master_filter['FCF/Assets Winsorized'] = \
    stats.mstats.winsorize(Data_for_Portfolio_master_filter['FCF/Assets'], \
                           limits=Winsorize_Threshold)
Data_for_Portfolio_master_filter['ROA Winsorized'] = \
    stats.mstats.winsorize(Data_for_Portfolio_master_filter['ROA'], \
                           limits=Winsorize_Threshold)
Data_for_Portfolio_master_filter['ROIC Winsorized'] = \
    stats.mstats.winsorize(Data_for_Portfolio_master_filter['ROIC'], \
                            limits=Winsorize_Threshold)
Data_for_Portfolio_master_filter['Gross Margin Winsorized'] = \
    stats.mstats.winsorize(Data_for_Portfolio_master_filter['GROSS MARGIN'], \
                           limits=Winsorize_Threshold)
Data_for_Portfolio_master_filter['Current Ratio Winsorized'] = \
    stats.mstats.winsorize(Data_for_Portfolio_master_filter['CURRENT RATIO'], \
                           limits=Winsorize_Threshold)
Data_for_Portfolio_master_filter['Interest/EBITDA Winsorized'] = \
    stats.mstats.winsorize(Data_for_Portfolio_master_filter['INTEREST/EBITDA'], \
                           limits=Winsorize_Threshold)

#create Z score

Data_for_Portfolio_master_filter['FCF/Assets Z score'] = \
    stats.zscore(Data_for_Portfolio_master_filter['FCF/Assets Winsorized'])
Data_for_Portfolio_master_filter['ROA Z score'] = \
    stats.zscore(Data_for_Portfolio_master_filter['ROA Winsorized'])
Data_for_Portfolio_master_filter['ROIC Z score'] = \
    stats.zscore(Data_for_Portfolio_master_filter['ROIC Winsorized'])
Data_for_Portfolio_master_filter['Gross Margin Z score'] = \
    stats.zscore(Data_for_Portfolio_master_filter['Gross Margin Winsorized'])
Data_for_Portfolio_master_filter['Current Ratio Z score'] = \
    stats.zscore(Data_for_Portfolio_master_filter['Current Ratio Winsorized'])
Data_for_Portfolio_master_filter['Interest/EBITDA Z score'] = \
    stats.zscore(Data_for_Portfolio_master_filter['Interest/EBITDA Winsorized'])

Data_for_Portfolio_master_filter['Quality Score'] = \
    Data_for_Portfolio_master_filter['FCF/Assets Z score'] \
        + Data_for_Portfolio_master_filter['ROA Z score'] \
        + Data_for_Portfolio_master_filter['ROIC Z score']\
        + Data_for_Portfolio_master_filter['Gross Margin Z score']\
        + Data_for_Portfolio_master_filter['Current Ratio Z score']\
        - Data_for_Portfolio_master_filter['Interest/EBITDA Z score']

###### SHAREHOLDER YIELD FACTOR #####

Data_for_Portfolio_master_filter['Shareholder Yield Winsorized'] = \
    stats.mstats.winsorize(Data_for_Portfolio_master_filter['Shareholder Yield'], \
                           limits=Winsorize_Threshold)
Data_for_Portfolio_master_filter['Shareholder Yield Z score'] = \
    stats.zscore(Data_for_Portfolio_master_filter['Shareholder Yield Winsorized'])
Data_for_Portfolio_master_filter['Shareholder Yield Score'] = \
    Data_for_Portfolio_master_filter['Shareholder Yield Z score'] 

###### LOW VOLATILITY FACTOR ######

#must have fundamental data from previous factors for price based factors
#as some equities have price data and no fundamental data which should not
#be included
Sector_stocks_Fundamental_tickers = Data_for_Portfolio_master_filter['ticker'].tolist()

Sector_stock_prices_vol_df = Sector_stock_prices.loc\
    [Sector_stock_prices['ticker'].isin(Sector_stocks_Fundamental_tickers)]

Sector_stock_prices_vol_df_1 = Sector_stock_prices_vol_df.iloc[:, [0, 1, 5]]

Sector_stock_prices_vol_df_1_wide = Sector_stock_prices_vol_df_1.pivot\
    (index='date', columns='ticker', values='close')

Sector_stock_prices_vol_df_1_wide = Sector_stock_prices_vol_df_1_wide.fillna(0)
Sector_stock_returns =  Sector_stock_prices_vol_df_1_wide.pct_change()      

#create rolling vol metric for previous 2 years
Sector_stock_rolling_vol = Sector_stock_returns.rolling(252*2).std()

#Choose second to last trading day to look at previous vol   
#Sometimes the dates are off when trying to line up end of quarter and business
#days so to eliminate errors in the for loop I go to day of quarter, shift forward
#a business day and then go back two business days
Date_to_execute_trade = pd.to_datetime(Date) + pd.tseries.offsets.QuarterEnd()
Date_to_execute_trade_plus1 = Date_to_execute_trade + pd.tseries.offsets.BusinessDay(1)
final_trade_date = Date_to_execute_trade_plus1 - pd.tseries.offsets.BusinessDay(2)

#pick the final trade date volatility for each ticker
Filter_Date_Vol = final_trade_date.strftime('%Y-%m-%d')
Filter_Vol_Signal = Sector_stock_rolling_vol.loc[Filter_Date_Vol]
Filter_Vol_Signal_Sort = Filter_Vol_Signal.sort_values().dropna()

#create z score and rank for the Volatility Factor
frame = { 'Vol': Filter_Vol_Signal_Sort} 
Filter_Vol_Signal_df = pd.DataFrame(frame)
Filter_Vol_Signal_df['Vol Z Score'] = stats.zscore(Filter_Vol_Signal_Sort)
Filter_Vol_Signal_df = Filter_Vol_Signal_df.reset_index()

Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.merge(Filter_Vol_Signal_df, how = 'inner', on = ['ticker']) 

###### TREND FACTOR #####

tickers_trend = list(Sector_stock_prices_vol_df_1_wide.columns)

#This is a very simply way to see how much a stock is in a trend up or down
#You could easily make this more complex/robust but it would cost you in 
#execution time
df_sma_50 = Sector_stock_prices_vol_df_1_wide.rolling(50).mean()
df_sma_100 = Sector_stock_prices_vol_df_1_wide.rolling(100).mean()
df_sma_150 = Sector_stock_prices_vol_df_1_wide.rolling(150).mean()
df_sma_200 = Sector_stock_prices_vol_df_1_wide.rolling(200).mean()

#Get the same date for vol measurement near rebalance date
Filter_Date_Trend = final_trade_date.strftime('%Y-%m-%d')
Filter_Trend_Signal_50 = df_sma_50.loc[Filter_Date_Trend]
Filter_Trend_Signal_100 = df_sma_100.loc[Filter_Date_Trend]
Filter_Trend_Signal_150 = df_sma_150.loc[Filter_Date_Trend]
Filter_Trend_Signal_200 = df_sma_200.loc[Filter_Date_Trend]

Price_Signal = Sector_stock_prices_vol_df_1_wide.loc[Filter_Date_Trend]

Filter_SMA_Signal_df = pd.DataFrame(tickers_trend)
Filter_SMA_Signal_df = Filter_SMA_Signal_df.rename(columns={0: "ticker"})
Filter_SMA_Signal_df['SMA 50 position'] = np.where(Price_Signal > Filter_Trend_Signal_50,1,0)
Filter_SMA_Signal_df['SMA 100 position'] = np.where(Price_Signal > Filter_Trend_Signal_100,1,0)
Filter_SMA_Signal_df['SMA 150 position'] = np.where(Price_Signal > Filter_Trend_Signal_150,1,0)
Filter_SMA_Signal_df['SMA 200 position'] = np.where(Price_Signal > Filter_Trend_Signal_200,1,0)
Filter_SMA_Signal_df['Trend Score'] = np.mean(Filter_SMA_Signal_df, axis=1)
Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.merge(Filter_SMA_Signal_df[['ticker','Trend Score']], how = 'inner', on = ['ticker'])

###### MOMENTUM FACTOR #####

tickers_momentum = list(Sector_stock_prices_vol_df_1_wide.columns)
#from the academic literature of 12 months - 1 month momentum 
df_mom_11_months = Sector_stock_prices_vol_df_1_wide.pct_change(22*11)

Filter_Date_Mom = Date_to_execute_trade_plus1 - pd.tseries.offsets.BusinessDay(24)
Filter_Date_Mom_trim = final_trade_date.strftime('%Y-%m-%d')
Filter_Mom_Signal = df_mom_11_months.loc[Filter_Date_Mom_trim]

Filter_MOM_df = pd.DataFrame(tickers_momentum)
Filter_MOM_df = Filter_MOM_df.rename(columns={0: "ticker"})
Filter_MOM_df['Percent Change'] = Filter_Mom_Signal.values

Filter_MOM_df = Filter_MOM_df.replace([np.inf, -np.inf], np.nan)
Filter_MOM_df = Filter_MOM_df.dropna()
Filter_MOM_df['Momentum Score'] = stats.zscore(Filter_MOM_df['Percent Change'])

Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.merge(Filter_MOM_df[['ticker','Momentum Score']], how = 'inner', on = ['ticker'])

C:\Users\DELL\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [80]:
Data_for_Portfolio_master_filter['Total Score'] = \
    Data_for_Portfolio_master_filter['Valuation Score'] +  \
    Data_for_Portfolio_master_filter['Quality Score'] + \
    Data_for_Portfolio_master_filter['Shareholder Yield Score'] -  \
    Data_for_Portfolio_master_filter['Vol Z Score'] * \
    (Data_for_Portfolio_master_filter['Momentum Score'] + \
    Data_for_Portfolio_master_filter['Trend Score'])

number_firms = Data_for_Portfolio_master_filter.shape
number_firms = number_firms[0]

In [81]:
Data_for_Portfolio_master_filter = \
    Data_for_Portfolio_master_filter.sort_values('Total Score', ascending=False)
Test=Data_for_Portfolio_master_filter[Data_for_Portfolio_master_filter['Total Score']>=8]

In [143]:
Data_for_Portfolio_master_filter[['ticker','sector','Total Score']].drop_duplicates(['ticker'],keep = 'first')

,ticker,sector,Total Score
166,BIIB,Healthcare,15.247131
1376,LPX,Basic Materials,14.702598
692,MU,Technology,14.396380
345,EBAY,Technology,13.901160
917,SWKS,Technology,13.690754
...,...,...,...
1393,NTRA,Healthcare,-12.783486
1403,NVAX,Healthcare,-13.100797
1378,RUN,Technology,-15.629291
1315,LYFT,Technology,-18.375465


In [139]:
my_stocks = ['EBAY','KNX','MNST','ZM'] 
Data_for_Portfolio_master_filter[['ticker','sector','Total Score']][Data_for_Portfolio_master_filter['ticker'].isin(my_stocks)].drop_duplicates(['ticker'])

,ticker,sector,Total Score
345,EBAY,Technology,13.901160
662,MNST,Consumer Defensive,8.657840
592,KNX,Industrials,5.652692
1330,ZM,Communication Services,4.004370


In [ ]:
# Data_for_Portfolio_master_filter[['ticker','sector','Total Score']][Data_for_Portfolio_master_filter['calendardate']> '2020-03-31'].drop_duplicates(['ticker'],keep = 'first')

In [146]:
Test=Test.drop_duplicates(['ticker'])
Test['sector'].value_counts().to_frame().style.bar()

,sector
Technology,20
Healthcare,11
Consumer Cyclical,5
Basic Materials,2
Industrials,2
Energy,1
Consumer Defensive,1


In [148]:
First =  Data_for_Portfolio_master_filter[:40].drop_duplicates(['ticker'])
Last = Data_for_Portfolio_master_filter[-40:].drop_duplicates(['ticker'])

In [149]:
First_tickers = First['ticker'].tolist()
Last_tickers = Last['ticker'].tolist()

## Merge selected stocks with equity price data

In [150]:
Best = Sector_stock_prices.loc\
    [Sector_stock_prices['ticker'].isin(First_tickers)]
Best = Best.iloc[:, [0, 1, 5]]
Worst = Sector_stock_prices.loc\
    [Sector_stock_prices['ticker'].isin(Last_tickers)]
Worst = Worst.iloc[:, [0, 1, 5]]

In [151]:
Best.sort_values(by='date', ascending=True)

,ticker,date,close
13885369,VRTX,2012-01-03,32.23
3862567,LRCX,2012-01-03,36.75
14007894,RHI,2012-01-03,28.66
16261629,STLD,2012-01-03,14.03
13885524,WSM,2012-01-03,36.94
...,...,...,...
2331919,HOLX,2022-06-14,67.87
2308612,EBAY,2022-06-14,42.58
2109248,BIIB,2022-06-14,194.18
6816252,MU,2022-06-14,58.70


In [152]:
#add a quarter to reporting so no lookahead bias
Date_to_execute_trade = pd.to_datetime(Date) + pd.tseries.offsets.QuarterEnd()
Date_to_execute_trade_plus1 = Date_to_execute_trade + pd.tseries.offsets.BusinessDay(1)
final_trade_date = Date_to_execute_trade_plus1 - pd.tseries.offsets.BusinessDay(1)

#add 4 quarters to end so the rebalance will be annual
end_date = Date_to_execute_trade + pd.tseries.offsets.QuarterEnd(4)
final_trade_date_trim = final_trade_date.strftime('%Y-%m-%d')
end_date_trim = end_date.strftime('%Y-%m-%d')
start_date = final_trade_date_trim
end_date = end_date_trim


#make data from long format to wide and fill in Na's with O
Best_wide = Best.pivot\
    (index='date', columns='ticker', values='close')
Best_wide = \
    Best_wide.fillna(0)
Worst_wide = \
    Worst.pivot(index='date', columns='ticker', values='close')
Worst_wide = \
    Worst_wide.fillna(0)


In [153]:
if Best_wide.empty == True:
    days = price_index
    number_days = len(price_index)
    filler_returns = np.repeat(0, number_days)
    df = pd.DataFrame({'Days': days, 'Returns': filler_returns})
    df = df.set_index('Days')
    Best_wide = df
else:
    pass

if Worst_wide.empty == True:
    days = price_index
    number_days = len(price_index)
    filler_returns = np.repeat(0, number_days)
    df = pd.DataFrame({'Days': days, 'Returns': filler_returns})
    df = df.set_index('Days')
    Worst_wide = df
else:
    pass

In [154]:
start_date = str('2018-03-31')
start_date,end_date

('2018-03-31', '2022-06-30')

In [155]:
#pick out start date and end date for calculating equity returns
Best_wide.loc[start_date:end_date].head()
Best_wide = \
    Best_wide.loc[start_date:end_date]
Best_returns_daily = Best_wide.pct_change()


#if there are no assets then the Cheap_returns_daily become NaN and in that
#case we will need to by pass the normal operations and just keep the dataframe
x = np.logical_and\
    (Best_returns_daily.shape[1] == 1, Best_returns_daily.isnull().all() == True)

if x[0] == True:
    Best_returns_daily = Best_wide
else:
    #get rid of first NaN row
    Best_returns_daily = Best_returns_daily.dropna(how='all')

    #get rid of stocks that have no trading
    Best_returns_daily = Best_returns_daily.dropna(axis='columns')

column_length = Best_returns_daily.shape[1]



In [156]:
#pick out start date and end date for calculating equity returns
Worst_wide.loc[start_date:end_date].head()
Worst_wide = \
    Worst_wide.loc[start_date:end_date]
Worst_returns_daily = Worst_wide.pct_change()


#if there are no assets then the Cheap_returns_daily become NaN and in that
#case we will need to by pass the normal operations and just keep the dataframe
x = np.logical_and\
    (Worst_returns_daily.shape[1] == 1, Worst_returns_daily.isnull().all() == True)

if x[0] == True:
    Worst_returns_daily = Worst_wide
else:
    #get rid of first NaN row
    Worst_returns_daily = Worst_returns_daily.dropna(how='all')

    #get rid of stocks that have no trading
    Worst_returns_daily = Worst_returns_daily.dropna(axis='columns')

column_length = Worst_returns_daily.shape[1]


In [157]:
Best_returns_daily.to_csv('B.csv')  
Worst_returns_daily.to_csv('W.csv')  

In [165]:
Best_returns_daily.head(5)

ticker,AMGN,ANET,BIIB,BIO,BKNG,DKS,EBAY,EXEL,FFIV,HOLX,HPQ,INTU,LOGI,LPX,LRCX,META,MU,NTAP,NVDA,REGN,RHI,STLD,SWKS,TPL,TXN,VRTX,WBD,WSM
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-04-03,0.013068,0.000000,-0.001536,0.006993,0.018076,0.011670,0.006352,-0.009772,0.011584,0.003821,0.008515,0.003473,0.008630,0.033498,0.021148,0.004634,0.029764,0.024848,0.019434,0.004961,0.025052,0.028478,0.011395,0.007119,0.015201,0.013115,0.020853,0.005225
2018-04-04,0.031506,0.021506,0.022216,0.005660,0.001664,0.039337,0.004544,0.048402,0.007517,0.013322,0.010319,0.015135,-0.003588,0.018424,0.018690,-0.006470,0.035694,0.024410,0.003958,0.015503,-0.006280,0.008625,0.020486,0.009509,0.019867,0.023743,0.023150,0.023990
2018-04-05,-0.008760,0.014597,-0.027239,-0.012175,0.009696,0.012237,0.006032,-0.009413,-0.001185,0.004830,0.000464,-0.007050,0.003047,0.014405,-0.032975,0.027337,-0.066492,0.012719,-0.021482,-0.025967,0.011614,0.024752,-0.008331,0.009246,-0.018904,-0.027510,0.003993,0.014838
2018-04-06,-0.022442,-0.015300,-0.027662,-0.013618,-0.027584,-0.050886,-0.023482,-0.045701,-0.017663,-0.027236,-0.035731,-0.024152,-0.009114,-0.024439,-0.022408,-0.013430,-0.027689,0.003339,-0.032216,-0.027329,-0.035117,-0.045235,-0.032794,-0.020641,-0.027191,-0.021629,-0.000442,-0.032320
2018-04-09,0.008802,0.003015,-0.000505,0.002868,0.007539,0.012441,-0.005372,-0.067330,0.000497,0.002745,0.005294,0.010198,0.001394,-0.005416,0.012956,0.004644,-0.010318,0.007447,0.005414,0.000282,0.014348,0.004600,0.004814,0.020235,0.003519,0.016272,-0.003979,-0.016899


In [160]:
Best_returns_daily.columns

Index(['AMGN', 'ANET', 'BIIB', 'BIO', 'BKNG', 'DKS', 'EBAY', 'EXEL', 'FFIV',
       'HOLX', 'HPQ', 'INTU', 'LOGI', 'LPX', 'LRCX', 'META', 'MU', 'NTAP',
       'NVDA', 'REGN', 'RHI', 'STLD', 'SWKS', 'TPL', 'TXN', 'VRTX', 'WBD',
       'WSM'],
      dtype='object', name='ticker')

In [27]:
Worst_returns_daily.columns

Index(['ABC', 'ALK', 'ARMK', 'CNP', 'DRI', 'EIX', 'EXPE', 'FANG', 'FCX', 'GME',
       'HES', 'LNG', 'LYV', 'NCLH', 'NTNX', 'NTRA', 'NVAX', 'PCG', 'PLUG',
       'RUN', 'SLB', 'SNAP', 'TRGP', 'TSLA', 'VST', 'W', 'WOLF', 'WYNN'],
      dtype='object', name='ticker')